<h1> Datto RMM - User Audit Data from Website via Requests <h1>

In [ ]:
# general purpose
import time
import datetime as dt


# Spider
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import re


# Data Ingest and Export
import pandas as pd
import csv

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# import configparser for env secrets
from configparser import ConfigParser
config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# Create Bot Object

## Create Firefox Driver

In [ ]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

firefox_dev_binary = FirefoxBinary(r'C:/Program Files/Firefox Developer Edition/firefox.exe')
f_driver = webdriver.Firefox(firefox_binary=firefox_dev_binary, executable_path=r'D:/web_drivers/mozilla/geckodriver-v0.31.0-win64/geckodriver.exe')

## Create Chrome Driver

## Request Header Info (if needed to impersonate api browser calls)

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'

# construct req body
data = CaseInsensitiveDict()
data = ''

In [ ]:
baseUrl = 'https://concord.centrastage.net'
auth_url = f'https://auth.datto.com'
#home_url = f'{baseUrl}'
links = []

# Login to DattoRMM

In [ ]:
f_driver.get(auth_url)

In [ ]:
title = f_driver.find_element_by_xpath('//h1')
title.text

In [ ]:
login_txtbox = f_driver.find_element_by_xpath("//input[@id='form_username']")
login_txtbox.send_keys((config['dattormm']['username']),Keys.ENTER)

In [ ]:
pass_txtbox = f_driver.find_element_by_xpath("//input[@id='form_password']")
pass_txtbox.send_keys((config['dattormm']['password']),Keys.ENTER)

In [ ]:
time.sleep(3)

In [ ]:
mfa_txtbox = f_driver.find_element_by_id('authy_token')
mfa_txtbox.send_keys('071984', Keys.ENTER) # number entered manually for now /TODO: automate MFA

# Non-Compliance Summary Page

In [ ]:
projectUrl = f'{baseUrl}/csm/myaccount/activity'
f_driver.get(projectUrl)

In [ ]:
getUrl = 'https://concord-realtime.centrastage.net/notifications/db835c7e6d64f81de926a13e06b79fb3170d1ad46e804d807482e1e05e75d14e'
postUrl = 'https://concord.centrastage.net/csm/csv/download?&users=&dateRange=2020%2F09%2F01+7%3A56+-+2022%2F07%2F07+7%3A56&searchQuery=&_=1657205763926&action=activityAjax&controller=myaccount&max=50&offset=0&timeZone=Atlantic%2FReykjavik'

In [ ]:

with requests.Session() as s:
    download = s.get(getUrl)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

In [ ]:
my_list